## RAG with multiple data sources!

### Wikipedia tool

In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [2]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)
print(wiki.name)

wikipedia


## VectorDB with OllamaEmbeddings

In [4]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://docs.smith.langchain.com")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents, OllamaEmbeddings())
retriever=vectordb.as_retriever
retriever

<bound method VectorStore.as_retriever of <langchain_community.vectorstores.faiss.FAISS object at 0x000001A1DBE06270>>

### Creating retriever tool

In [5]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(retriever, "langsmith_search", "Search for information about langsmith search" )

In [6]:
retriever_tool.name

'langsmith_search'

### ARXIV Tool

In [7]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv=ArxivQueryRun(arxiv_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

#### combining all the tools in a list

In [32]:
tools = [wiki, arxiv, retriever_tool]

In [33]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\Admin\\Desktop\\GUVI\\LangChain\\.venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(),
 Tool(name='langsmith_search', description='Search for information about langsmith search', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001A194504FE0>, retriever=<bound method VectorStore.as_retriever of <langchain_community.vectorstores.faiss.FAISS object at 0x000001A1DBE06270>>, document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x000001A194505440>, retriever=<bound method VectorStore.as_retriever of <langchain_community.vectorstores.faiss.FAISS object at 0x000001A1DBE06270>>, document

In [72]:
from langchain_community.llms import Ollama
from dotenv import load_dotenv
import os

#Environment variables loaded
load_dotenv()

#LangSmith Tracking 
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")

# Ollama LLAMA2 LLM
llm=Ollama(model="llama2")

In [106]:
# Prompt tempalate

#from langchain import hub

#prompt = hub.pull("hwchase17/openai-tools-agent")
#prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

## Agents

In [124]:
from langchain.agents import create_react_agent
from langchain.prompts import PromptTemplate

# Assuming wiki, arxiv, and retriever_tool are defined as Tool objects
tools = [wiki, arxiv, retriever_tool]
tool_names = [tool.name for tool in tools]

prompt = PromptTemplate(
    input_variables=["agent_scratchpad", "tool_names", "tools", "user_input"],
    template="You have everything to respond to the user. {agent_scratchpad}. You have access to the following tools:{tools} {tool_names}.\nUse them to answer the user's query: {user_input}"
)

agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)


ValueError: Prompt missing required variables: {'tools'}

In [108]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-tools-agent', 'lc_hub_commit_hash': 

In [109]:
#Agent executer to run agent

from langchain.agents import AgentExecutor
agent_executer=AgentExecutor(agent=agent, tools=tools, verbose=True)

In [111]:
agent_executer.invoke({"input":"LangSmith"})



> Entering new AgentExecutor chain...


ValueError: This output parser only works on ChatGeneration output

In [112]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

result = agent_executor.invoke({"input": "LangSmith"})
print(result)




> Entering new AgentExecutor chain...


ValueError: This output parser only works on ChatGeneration output